# Proyecto: Predicción de Abandono de Clientes (Churn) - Beta Bank

## 1. Introducción y Contexto del Problema

En la industria bancaria, la retención de clientes es un factor crucial para la rentabilidad a largo plazo. Recientemente, los analistas de **Beta Bank** han detectado una tendencia preocupante: los clientes están cancelando sus cuentas y abandonando el banco poco a poco cada mes.

Tras un análisis financiero, los banqueros han llegado a una conclusión determinante: **es mucho más rentable retener a los clientes actuales que invertir recursos en atraer nuevos clientes.**

Para abordar este problema de manera proactiva, el banco ha recopilado un conjunto de datos históricos que refleja el comportamiento pasado de los clientes y la terminación de sus contratos.

## 2. Objetivo del Proyecto

El objetivo principal de este proyecto es **desarrollar un modelo de Machine Learning capaz de predecir si un cliente dejará el banco pronto** (*clasificación binaria*). Al identificar a los clientes con alta probabilidad de abandono (*churn*), el banco podrá tomar medidas preventivas, como ofrecer incentivos o promociones personalizadas, para intentar retenerlos.

Criterios de éxito:

- Se requiere construir un modelo que alcance un valor en la **métrica F1 de al menos 0.59** en el conjunto de prueba.

- Adicionalmente, se evaluará la métrica **AUC-ROC** para compararla con el valor F1 y determinar la capacidad predictiva general del modelo.

## 3. Metodología

Para cumplir con los objetivos trazados, el proyecto se dividirá en las siguientes fases:

**1- Preparación de los Datos:** Carga del dataset, limpieza, tratamiento de valores ausentes en variables clave (*como Tenure*) y codificación de variables categóricas.

**2- Análisis Inicial y Modelo Base:** Evaluación del equilibrio de clases en la variable objetivo y entrenamiento de un modelo inicial sin ajustes para establecer un punto de referencia.

**3- Mejora del Modelo (*Manejo del Desequilibrio*):** Aplicación de técnicas de corrección de desequilibrio de clases (como el ajuste de pesos y el sobremuestreo/upsampling) e iteración de hiperparámetros para encontrar el modelo con el mejor rendimiento en validación.

**4- Prueba Final:** Evaluación del mejor modelo seleccionado en un conjunto de datos de prueba independiente para verificar que se cumpla el umbral del valor F1 $\ge$ 0.59 y análisis de la métrica AUC-ROC.

### Paso 1: Descarga y Preparación de los Datos

En este primer paso nos encargaremos de cargar los datos, limpiarlos eliminando columnas que no sirven para predecir, tratar los valores nulos, transformar las variables de texto a números y finalmente, dividir y escalar los datos.

In [1]:
# Importamos librerias
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import f1_score, roc_auc_score

In [2]:
# Importamos la base de datos

df = pd.read_csv('/datasets/Churn.csv')

In [3]:
# Revisamos la informacion de la base de datos
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10000 entries, 0 to 9999
Data columns (total 14 columns):
 #   Column           Non-Null Count  Dtype  
---  ------           --------------  -----  
 0   RowNumber        10000 non-null  int64  
 1   CustomerId       10000 non-null  int64  
 2   Surname          10000 non-null  object 
 3   CreditScore      10000 non-null  int64  
 4   Geography        10000 non-null  object 
 5   Gender           10000 non-null  object 
 6   Age              10000 non-null  int64  
 7   Tenure           9091 non-null   float64
 8   Balance          10000 non-null  float64
 9   NumOfProducts    10000 non-null  int64  
 10  HasCrCard        10000 non-null  int64  
 11  IsActiveMember   10000 non-null  int64  
 12  EstimatedSalary  10000 non-null  float64
 13  Exited           10000 non-null  int64  
dtypes: float64(3), int64(8), object(3)
memory usage: 1.1+ MB


In [4]:
# Inspeccionamos la estructura e informacion de la base de datos
df.head()

,RowNumber,CustomerId,Surname,CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited
0,1,15634602,Hargrave,619,France,Female,42,2.0,0.00,1,1,1,101348.88,1
1,2,15647311,Hill,608,Spain,Female,41,1.0,83807.86,1,0,1,112542.58,0
2,3,15619304,Onio,502,France,Female,42,8.0,159660.80,3,1,0,113931.57,1
3,4,15701354,Boni,699,France,Female,39,1.0,0.00,2,0,0,93826.63,0
4,5,15737888,Mitchell,850,Spain,Female,43,2.0,125510.82,1,1,1,79084.10,0


Se tienen 10,000 registros, se cuentan con 14 columnas, solo la columnaa Tenure cuenta con valores ausentes, son de tipo object, numerico y flotanes.

**Objetivo:** Son 10,000 registros con tamaño ideal para predecir el abandono de clientes utilizando la columna Exited.

**Problema principal:** Le faltan 909 datos en la columna Tenure (tiempo de permanencia). Tienes que rellenar estos vacíos antes de aplicar cualquier modelo predictivo.

**Limpieza obligatoria:** Se deben borrar las columnas que no aportan valor predictivo (RowNumber, CustomerId, Surname), convertir las columnas de texto a formato numérico (Geography, Gender) y estandarizar las columnas con números grandes (Balance, EstimatedSalary).

In [5]:
# Limpieza Inicial
# Las columnas RowNumber, CustomerId y Surname no determinan si un cliente abandona el banco.
# Por lo tanto, las eliminamos para no confundir al modelo.
df = df.drop(['RowNumber', 'CustomerId', 'Surname'], axis=1)

In [6]:
#Tratamos los valores ausentes
# La columna 'Tenure' tiene valores nulos. Usamos la mediana para rellenarlos y no perder esos clientes.
df['Tenure'] = df['Tenure'].fillna(df['Tenure'].median())

In [7]:
# Codificamos de variables categoricas
# Convertimos 'Geography' (país) y 'Gender' (género) en variables numéricas (0 y 1).
# Usamos drop_first=True para evitar la redundancia de datos (trampa de las variables ficticias).
df = pd.get_dummies(df, drop_first=True)

In [8]:
# Dividision de datos
# Separamos las características (X) y el objetivo a predecir (y)
X = df.drop('Exited', axis=1)
y = df['Exited']

# Dividimos los datos en 3 partes: Entrenamiento (60%), Validación (20%) y Prueba (20%).
# Usamos stratify=y para que la proporción de "Exited" se mantenga igual en los 3 conjuntos.
X_train, X_temp, y_train, y_temp = train_test_split(X, y, test_size=0.4, random_state=12345, stratify=y)
X_valid, X_test, y_valid, y_test = train_test_split(X_temp, y_temp, test_size=0.5, random_state=12345, stratify=y_temp)

In [9]:
# Escalamos caracteristicas
# Estandarizamos las variables numéricas para que todas tengan un peso similar en el modelo.
numeric = ['CreditScore', 'Age', 'Tenure', 'Balance', 'NumOfProducts', 'EstimatedSalary']
scaler = StandardScaler()

# Ajustamos el escalador SOLO con los datos de entrenamiento
scaler.fit(X_train[numeric])

# Convierte las vistas en copias independientes
X_train = X_train.copy()
X_valid = X_valid.copy() 
X_test = X_test.copy()

# Transformamos los tres conjuntos
X_train[numeric] = scaler.transform(X_train[numeric])
X_valid[numeric] = scaler.transform(X_valid[numeric])
X_test[numeric] = scaler.transform(X_test[numeric])

**Conclusión:** Hemos transformado nuestros datos crudos en información limpia y numérica que un modelo de Machine Learning puede procesar. Al dividir los datos en tres conjuntos (entrenamiento, validación y prueba) y aplicar el escalado, garantizamos que el modelo aprenderá de forma correcta y podremos evaluarlo sin trampas ni fugas de datos.

### Paso 2: Examen del Equilibrio de Clases y Modelo Base (Desequilibrado)

Antes de hacer optimizaciones avanzadas, debemos entender cómo se distribuye lo que queremos predecir (los que se van vs. los que se quedan) y cómo se comporta un modelo si ignoramos esta distribución.

In [10]:
# 1- Analisis del equilibrio de clases
print("Proporción de clases en el objetivo (y):")
print(y.value_counts(normalize=True))
# El resultado mostrará aprox. 80% (0: Se quedan) y 20% (1: Se van)

#  2- Entrenamiento de modelo sin correcciones
# Entrenamos un Random Forest probando diferentes hiperparámetros.
best_rf_imbalanced = None
best_f1_imbalanced = 0

# Iteramos sobre la cantidad de árboles (estimadores) y la profundidad de cada árbol
for est in range(10, 51, 10):
    for depth in range(1, 15):
        model = RandomForestClassifier(random_state=12345, n_estimators=est, max_depth=depth)
        model.fit(X_train, y_train)
        predictions_valid = model.predict(X_valid)
        
        # Evaluamos con la métrica F1
        f1 = f1_score(y_valid, predictions_valid)
        if f1 > best_f1_imbalanced:
            best_f1_imbalanced = f1
            best_rf_imbalanced = model

# Calculamos AUC-ROC usando las probabilidades (predict_proba)
prob_valid = best_rf_imbalanced.predict_proba(X_valid)[:, 1]
auc_roc = roc_auc_score(y_valid, prob_valid)

print(f"\nMétodo Desequilibrado - F1 (Validación): {best_f1_imbalanced:.4f}")
print(f"Método Desequilibrado - AUC-ROC: {auc_roc:.4f}")

Proporción de clases en el objetivo (y):
0    0.7963
1    0.2037
Name: Exited, dtype: float64

Método Desequilibrado - F1 (Validación): 0.6112
Método Desequilibrado - AUC-ROC: 0.8696


**Conclusión:**
Descubrimos que las clases están muy desequilibradas (80% vs 20%). Al entrenar el modelo en estas condiciones, el valor F1 en validación llega apenas a ~0.61. Aunque no es terrible, el modelo tiende a predecir "0" (que el cliente se queda) la mayoría de las veces por ser la clase dominante, siendo ineficiente para encontrar a los clientes que realmente se van.

### Paso 3: Mejora de la Calidad del Modelo (Corregir el Desequilibrio)

Para que el modelo preste más atención a la clase minoritaria (los que se van), utilizaremos dos técnicas distintas y buscaremos los mejores hiperparámetros para cada una.

In [11]:
# ==========================================
# Tecnica 1: Ajuste de pesos (CLASS WEIGHT)
# ==========================================
best_rf_balanced = None
best_f1_balanced = 0

for est in range(10, 101, 10):
    for depth in range(1, 15):
        # Añadimos class_weight='balanced' para penalizar más los errores en la clase "1"
        model = RandomForestClassifier(random_state=12345, n_estimators=est, max_depth=depth, class_weight='balanced')
        model.fit(X_train, y_train)
        predictions_valid = model.predict(X_valid)
        f1 = f1_score(y_valid, predictions_valid)
        
        if f1 > best_f1_balanced:
            best_f1_balanced = f1
            best_rf_balanced = model

prob_valid_bal = best_rf_balanced.predict_proba(X_valid)[:, 1]
auc_roc_bal = roc_auc_score(y_valid, prob_valid_bal)

print(f"Técnica 1 (Pesos) - F1: {best_f1_balanced:.4f} | AUC-ROC: {auc_roc_bal:.4f}")


# ==========================================
# Tecnica 2: Sobremuestreo (UPSAMPLING)
# ==========================================
# Función para multiplicar las filas de la clase minoritaria
def upsample(features, target, repeat):
    features_zeros = features[target == 0]
    features_ones = features[target == 1]
    target_zeros = target[target == 0]
    target_ones = target[target == 1]

    # Multiplicamos la clase "1" por el factor 'repeat'
    features_upsampled = pd.concat([features_zeros] + [features_ones] * repeat)
    target_upsampled = pd.concat([target_zeros] + [target_ones] * repeat)
    
    return features_upsampled, target_upsampled

# Como la clase 0 es ~4 veces mayor que la 1, repetimos la clase 1 unas 4 veces
X_upsampled, y_upsampled = upsample(X_train, y_train, 4)

best_rf_up = None
best_f1_up = 0

for est in range(10, 101, 10):
    for depth in range(1, 15):
        # Entrenamos con el nuevo set de datos sobremuestreado
        model = RandomForestClassifier(random_state=12345, n_estimators=est, max_depth=depth)
        model.fit(X_upsampled, y_upsampled)
        predictions_valid = model.predict(X_valid)
        f1 = f1_score(y_valid, predictions_valid)
        
        if f1 > best_f1_up:
            best_f1_up = f1
            best_rf_up = model

prob_valid_up = best_rf_up.predict_proba(X_valid)[:, 1]
auc_roc_up = roc_auc_score(y_valid, prob_valid_up)

print(f"Técnica 2 (Upsampling) - F1: {best_f1_up:.4f} | AUC-ROC: {auc_roc_up:.4f}")

Técnica 1 (Pesos) - F1: 0.6499 | AUC-ROC: 0.8698
Técnica 2 (Upsampling) - F1: 0.6538 | AUC-ROC: 0.8758


**Conclusión:**
Al aplicar técnicas para corregir el desequilibrio, nuestro modelo mejoró sustancialmente. Ambas técnicas (*class_weight='balanced' y Upsampling*) lograron subir el F1 a aproximadamente 0.65 en el conjunto de validación, dándonos seguridad de que el modelo ahora sí identifica correctamente a los clientes propensos a irse. Seleccionaremos el modelo entrenado con Upsampling (*best_rf_up*) para nuestra prueba final por su robustez.

### Paso 4: Prueba Final del Modelo

Ya tenemos nuestro modelo definitivo y optimizado. Ahora lo pondremos a prueba frente al conjunto de datos de Prueba (Test), datos que el modelo jamás ha visto.

In [12]:
# Evaluacion Final
# Usamos el mejor modelo guardado (best_rf_up) para predecir sobre X_test
predictions_test = best_rf_up.predict(X_test)

# Obtenemos las probabilidades de que la clase sea "1" (se va) para el AUC-ROC
prob_test = best_rf_up.predict_proba(X_test)[:, 1]

# Calculamos las métricas definitivas
test_f1 = f1_score(y_test, predictions_test)
test_auc_roc = roc_auc_score(y_test, prob_test)

print("=== RESULTADOS DE LA PRUEBA FINAL ===")
print(f"Métrica F1 (Conjunto de Prueba): {test_f1:.4f}")
print(f"Métrica AUC-ROC (Conjunto de Prueba): {test_auc_roc:.4f}")

=== RESULTADOS DE LA PRUEBA FINAL ===
Métrica F1 (Conjunto de Prueba): 0.5996
Métrica AUC-ROC (Conjunto de Prueba): 0.8522


**Conclusión:**
El modelo logró una puntuación F1 aproximada de 0.60, superando con éxito el umbral exigido para aprobar el proyecto (0.59). Además, el valor de la métrica AUC-ROC es de ~0.85, lo que significa que la capacidad general del modelo para distinguir entre los clientes leales y los que abandonarán el banco es muy alta (mucho mejor que el 0.5 de un modelo aleatorio).

Hemos cumplido los objetivos del negocio: crear una herramienta confiable y probada para identificar a los clientes en riesgo de fuga para que los banqueros actúen y los salven.

## Conclucion General Impacto para el Negocio:

El modelo resultante proporciona a Beta Bank una herramienta predictiva robusta y confiable. Al poder identificar de manera anticipada a los clientes con alta probabilidad de abandonar la institución, **el banco ahora puede optimizar sus recursos financieros, dirigiendo campañas de retención**, ofertas personalizadas y un mejor servicio al cliente específicamente a este segmento en riesgo, logrando así el objetivo financiero principal: ahorrar dinero al retener a los clientes actuales en lugar de gastar en la adquisición de nuevos.